# Embeddings
https://platform.openai.com/docs/models#embeddings

임베딩(Embeddings)은 텍스트를 수치적으로 표현한 값으로, 두 텍스트 간의 연관성을 측정하는 데 사용된다.

임베딩은 검색, 군집화(clustering), 추천 시스템, 이상 탐지, 분류와 같은 작업에 유용하다.

**모델 및 출력 차원**

| 모델 이름                     | 설명                                                              | 출력 차원 |
|-------------------------------|-------------------------------------------------------------------|-----------|
| **text-embedding-3-large**   | 영어 및 비영어 작업 모두에서 가장 강력한 성능을 가진 모델           | 3,072     |
| **text-embedding-3-small**   | 2세대 ada 임베딩 모델보다 성능이 향상된 모델                        | 1,536     |
| **text-embedding-ada-002**   | 1세대 모델 16개를 대체하는 가장 강력한 2세대 임베딩 모델             | 1,536     |


## MTEB Leaderboard
**Massive Text Embedding Benchmark (MTEB) Leaderboard**

https://huggingface.co/spaces/mteb/leaderboard

**MTEB Leaderboard**는 Hugging Face에서 제공하는 벤치마크 리더보드 페이지로, 다양한 언어 모델(Language Model)과 임베딩 모델(Embedding Model)의 성능을 객관적으로 비교·평가하는 공간이다.

**MTEB Leaderboard에서 순위 산정 방식**

**MTEB Leaderboard**의 순위는 다양한 자연어 처리 태스크(분류, 클러스터링, 검색, 문장 유사도 등)에서 모델이 얻은 점수들의 평균을 기반으로 산정된다. 즉, 여러 벤치마크 데이터셋에서 모델의 성능을 측정하고, 이를 종합하여 평균 점수를 계산한 뒤, 이 평균 점수가 높은 순서대로 모델이 정렬된다.

**주요 평가 방식**

- **평가 태스크 종류**
  - 분류(Classification): F1 점수
  - 클러스터링(Clustering): V-measure
  - 쌍 분류(Pair Classification): Average Precision
  - 재정렬(Reranking): MRR@k, MAP
  - 검색(Retrieval): nDCG@k
  - 의미 유사도(STS): Spearman correlation
  - 요약(Summarization): Spearman correlation  
  각 태스크별로 대표적인 평가 지표가 다르며, 모델은 여러 태스크에서 평가를 받는다[2].

- **평균 점수 산정**
  - 각 태스크별로 모델이 얻은 점수를 모두 합산한 뒤, 태스크 수로 나누어 평균 점수를 구한다.
  - 이 평균 점수가 리더보드의 기본 순위 기준이 된다.

- **부분 평가 가능**
  - 모든 태스크를 수행하지 않아도 특정 태스크만 평가받아 부분 리더보드에 오를 수 있다. 예를 들어, 클러스터링 태스크만 평가받아 해당 부분 순위에 표시될 수 있다.
  

In [18]:
from openai import OpenAI                 # OpenAI API를 사용하기 위한 클라이언트 클래스
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv('openai_key')
client = OpenAI(api_key = api_key)


In [19]:
response = client.embeddings.create(
    model = 'text-embedding-3-small',
    input = ["Hello world","안녕하십니까?"] # 임베딩으로 변환할 문장 리스트
)

response

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.002078542485833168, -0.04908587411046028, 0.020946789532899857, 0.03135102614760399, -0.04530530795454979, -0.026402482762932777, -0.028999701142311096, 0.06030462309718132, -0.02571091614663601, -0.01482258178293705, 0.015444992110133171, -0.029983261600136757, -0.020393535494804382, -0.03334888815879822, 0.025833861902356148, 0.014207855798304081, -0.0700787678360939, 0.012432834133505821, 0.014791845344007015, 0.048839982599020004, 0.02073163539171219, -0.008890475146472454, -0.015114576555788517, -0.01661297120153904, 0.025926070287823677, -0.002902659587562084, -0.024327782914042473, 0.024281678721308708, 0.0017433245666325092, -0.055724915117025375, 0.02308296225965023, -0.045489728450775146, -0.008652268908917904, 0.0031619970686733723, 0.004583551082760096, 0.001794231589883566, 0.02669447846710682, 0.010158347897231579, -0.012056314386427402, -0.011472324840724468, -0.01491479016840458, -0.023129066452383995, 0.025357449427

In [20]:
emb_voc0 = response.data[0].embedding   # 첫 번쨰 문장의 임베딩 벡터
emb_voc1 = response.data[1].embedding

print(len(emb_voc0))    # 첫 번쨰 임베딩 벡터 차원(길이)
print(len(emb_voc1))

1536
1536


In [21]:
# 여러 문장을 한 번에 임베딩으로 변환하고, 각 문장의 벡터를 리스트로 변환하는 함수
def text_to_embedding(texts,model='text-embedding-3-small'):
    texts = [text.replace('\n',' ') for text in texts]          # 개행 (\n)을 공백으로 치환해 임베딩 입력 정리
    response = client.embeddings.create(model=model, input=texts) # 임베딩 생성 요청
        
    return [data.embedding for data in response.data]      # 각 문장의 임베딩 벡터만 추출해 리스트로 변환
    
    
vecs = text_to_embedding(['Hello world', '안녕하세요'])     # 2개 문장을 임베딩으로 변환

print(len(vecs[0]))
print(len(vecs[1]))

1536
1536


## 음식리뷰 임베딩처리

https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

corpus를 임베딩 벡터로 변환하고, 이후에 유사도를 통한 검색이 가능하다.

In [25]:
import pandas as pd

# CSV 로드( 0번 컬럼은 인덱스로 사용)
review_df = pd.read_csv('fine_food_reviews_1k.csv', index_col=0)
review_df.info()    # 컬럼 타입/결측치/행 수 등 메타정보
review_df.head()    # 상위 5개 행

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Time       1000 non-null   int64 
 1   ProductId  1000 non-null   object
 2   UserId     1000 non-null   object
 3   Score      1000 non-null   int64 
 4   Summary    1000 non-null   object
 5   Text       1000 non-null   object
dtypes: int64(2), object(4)
memory usage: 54.7+ KB


,Time,ProductId,UserId,Score,Summary,Text
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...


In [28]:
review_df = review_df[['Summary', 'Text']]

# Summary와 Text 공백 제거 후 하나의 문자열로 결합
review_df['Content'] = 'Title: ' + review_df['Summary'].str.strip() + "; Content:" + review_df['Text'].str.strip()
review_df.head()

,Summary,Text,Content
0,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content:Not pleased ...
2,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ...."
3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...
4,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content:My dog ...


In [ ]:
# Content를 리스트로 변환 임베딩 생성 -> embedding 컬럼에 저장
review_df['embedding'] = text_to_embedding(review_df['Content'].tolist())
review_df.head(3)

,Summary,Text,Content,embedding
0,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[0.03539932146668434, -0.021147025749087334, -..."
1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content:Not pleased ...,"[0.010457453317940235, 0.03552861511707306, -0..."
2,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[0.0031563059892505407, 0.013021500781178474, ..."


In [ ]:
embed_df  = review_df['embedding'].to_frame("embedding")    # Series(embdding) -> DataFrame으로 변환
embed_df.index = review_df['Content']                       # 인덱스는 Content 텍스트로 설정
embed_df

,embedding
Content,
Title: where does one start...and stop... with a treat like this; Content:Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,"[0.03539932146668434, -0.021147025749087334, -..."
"Title: Arrived in pieces; Content:Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.","[0.010457453317940235, 0.03552861511707306, -0..."
"Title: It isn't blanc mange, but isn't bad . . .; Content:I'm not sure that custard is really custard without eggs. But this comes close. I got it for use in a ""Vegan pancake"" recipe. We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here. One of the cooking/recipe sites had a recipe using this and there were lots of great reviews. I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the so-called custard isn't so bad. I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor). It's fun playing with it. You could dress it up with fruit. Seems to come out on the thin side when you make it as directed, so I use less milk because I like my custards to set firm. As a custard sauce it's fine. I would say it tastes something between a pudding and a custard.<br /><br />If you want a really good egg-free ""custard"" get an original recipe for ""blanc mange."" It takes a lot longer to make, but it's certainly worth the difference.","[0.0031563059892505407, 0.013021500781178474, ..."
"Title: These also have SALT and it's not sea salt.; Content:I like the fact that you can see what you're getting and that there are no bones or dark meat. There are 7 nice big chunks in every jar.<br /><br />These taste like tuna in a can but, because they're preserved in glass, you don't have to worry about either aluminum or BPA; BUT ... they are not just tuna and spring water.<br /><br />There is salt in there, too, and it's not healthy sea salt, it's toxic table salt.<br /><br />I am trying to contact Tonnino to confirm that. I might be wrong because the label states that the ingredients are ""tuna fish"" but the sticker on the top clarifies that it is the smaller (healthier) yellowfin, so the ""salt"" listed in the ingredients might be sea salt but, if it was, why don't they say so?<br /><br />Without confirmation, I will continue to look for a salt-free olive-oil free tuna preserved in glass.<br /><br />If you know of one, please contact me!","[-0.003678289009258151, 0.014659896492958069, ..."
Title: Happy with the product; Content:My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,"[0.012326869182288647, -0.05477552115917206, 0..."
...,...
Title: Delicious!; Content:I have ordered these raisins multiple times. They are always great and arrive timely. I can't go back to store bought chocolate covered raisins now! Love this product.,"[0.014931892044842243, -0.039352718740701675, ..."
Title: Good Training Treat; Content:My dog will come in from outside when I am training her and look at the cupboard waiting for her treat. When I use the clicker training method she comes because she knows she has something special.,"[-0.022726766765117645, -0.011521120555698872,..."
Title: Jamica Me Crazy Coffee; Content:Wolfgang Puck's Jamaica Me Crazy is that wonderful blend of island flavors in a coffee. Have loved it from the first time tasting. Great product.,"[-0.029734576120972633, -0.046856801956892014,..."


### 유사도 검색
사용자가 입력한 쿼리와 유사한 리뷰를 조회할 수 있다.(벡터 서치 기반)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 쿼리와 가장 유사한 리뷰를 벡터 유사도로 Top-N개 검색하는 함수
def review_vector_search(query, embed_df=embed_df, top_n=5):
    query_emb = text_to_embedding([query])  # 사용자 쿼리를 임베딩 벡터로 변환
    
    df = embed_df.copy()    # 원본 보호를 위한 복사
    df['cos_sim'] = df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0,0])
    
    df = df.sort_values('cos_sim', ascending=False).head(top_n) # 유하도가 높은 순으로 정렬 후 Top-N개 선택
    df = df.reset_index()           # 인덱스(Content)를 컬럼으로 변환
    df = df[['Content','cos_sim']]  # Top-N 검색 결과 DataFrame 생성
    return df

pd.set_option('display.max_colwidth', None) # Content 컬럼이 잘리는 것을 방지하기 위해 최대 출력 길이 제한 해제(default 50)
review_vector_search('delicious fruit')     # delicious fruit과 유사한 리뷰 Top-5 검색

cosine_similarity([emb])  
[emb] = [[0.1221121,..]] => (1,d)  
query_emb = (1,d)  
cosine similarity 계산시 [[0.955....]] => (1,1)  
[0, 0] => 값만 빼온다.

In [41]:
review_vector_search('best coffee')

,Content,cos_sim
0,"Title: Best coffee ever!; Content:In my opinion this is the best coffee ever! I've been drinking coffee for 50 plus years and this is what I serve to myself and friends. However, I wish I could find this grind in a pound size, so I could make a full pot rather than just a single cup.",0.608850
1,"Title: Great Coffee; Content:I have a coffee maker that grinds my coffee beans. It's hard to find whole bean decafinated coffee. When I find it in the brand that I like, I am excited. Seattle's Best is my favorite.",0.605922
2,"Title: Better than you-know-who's coffee...; Content:So my wife is a latte freak, and nursing, so decaf is the approved type. After the Senseo left the market, I struggled and found the <a href=""http://www.amazon.com/gp/product/B0047BIWSK"">Aerobie AeroPress Coffee and Espresso Maker</a> which is like a French Press for the 21st century. After getting our recipe figured out, my wife, who's been buying Venti Decaf Latte's at $4 a pop almost daily for years now declares that Seattle's best Level 3 Decaf in her home-made Latte is the best coffee she can get. We've tried other bands, and this is her favorite, hands down!",0.604539
3,Title: Good coffee.; Content:This is the best Donut Shop Blend out there. Though most k-cups taste a little freeze dried. Keurig needs to work on that. I prefer San Francisco Bay Coffee Company.,0.600996
4,Title: super coffee; Content:Great coffee and so easy to brew. This coffee has great aroma and is good to the last drop. I actually like all the brands. This is the way coffee should taste!!,0.596348
